<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/7%20-%20Topic%20modelling/topic_modelling_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic modelling con Gensim e NLTK
In questo notebook identificheremo i topic di titoli di notizie creando un modello **latent dirichlet allocation (LDA)** con Gensim, una popolare libreria Python per la modellazione degli argomenti.

## Otteniamo il dataset
Il dataset contenente un milione di titoli di giornale è presente su Kaggle [a questo indirizzo](https://www.kaggle.com/therohk/million-headlines), per scaricare l'ultima versione aggiornata devi registrarti su Kaggle. Se non vuoi registrarti al momento puoi scaricare una versione più vecchia del dataset da [questo indirizzo ](https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv). Se utilizzi Google Colab o comunque hai wget installato sul tuo computer esegui pure la cella di codice qui sotto per scaricare il dataset.

In [1]:
!wget https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv

--2019-04-17 09:47:53--  https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54096356 (52M) [text/plain]
Saving to: ‘abcnews-date-text.csv’

abcnews-date-text.c 100%[===================>]  51.59M   126MB/s    in 0.4s    

2019-04-17 09:47:59 (126 MB/s) - ‘abcnews-date-text.csv’ saved [54096356/54096356]



Il dataset è un file CSV, carichiamo all'interno di un DataFrame con pandas e teniamo soltanto il 20% delle righe.

In [4]:
import pandas as pd

headlines_df = pd.read_csv("abcnews-date-text.csv")
headlines_df = headlines_df.sample(frac=.2, random_state=0)
print(headlines_df.shape)
headlines_df.head()

(216434, 2)


,publish_date,headline_text
131253,20041202,labor attacks nationals mp over milk link
993448,20151109,coraki public school hip hop video
691669,20120501,dairy record
841363,20131211,sa country hour podcast 11 december
92568,20040525,duffy given ireland chance against springboks


## Preprocessing del testo
Preprocessiamo il testo, rimuovendo le stop words e eseguendo la lemmatizzazione per ridurre la dimensione del nostro vocabolario. Per la lemmatizzazione useremo NLTK, importiamolo e scarichiamo il modulo 'wordnet'.

In [5]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Definiamo la funzione che utilizzeremo per preprocessare la singola frase, per estrarre i tokens possiamo usare la funzione simple_preprocess di gensim, anche per le stop words possiamo usare gensim, che ne mette a disposizione ben 337.

In [0]:
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess

lemmatizer = nltk.stem.WordNetLemmatizer()

def preprocess(text):
  
  tokens = []
  
  for token in simple_preprocess(text):
    if(token not in STOPWORDS):
      tokens.append(lemmatizer.lemmatize(token, pos='v'))
  
  return tokens

Applichiamo la funzione appena creata ad ogni titolo nel DataFrame utilizzando la funzione map e creiamo una nuova colonna con i titoli processati.

In [14]:
headlines_df["headline_processed"] = headlines_df["headline_text"].map(preprocess)
headlines_df.head()

,publish_date,headline_text,headline_processed
131253,20041202,labor attacks nationals mp over milk link,"[labor, attack, nationals, mp, milk, link]"
993448,20151109,coraki public school hip hop video,"[coraki, public, school, hip, hop, video]"
691669,20120501,dairy record,"[dairy, record]"
841363,20131211,sa country hour podcast 11 december,"[sa, country, hour, podcast, december]"
92568,20040525,duffy given ireland chance against springboks,"[duffy, give, ireland, chance, springboks]"


Estraiamo i titoli preprocessati all'interno di un'array numpy.

In [16]:
headlines = headlines_df["headline_processed"].values
headlines[:3]

array([list(['labor', 'attack', 'nationals', 'mp', 'milk', 'link']),
       list(['coraki', 'public', 'school', 'hip', 'hop', 'video']),
       list(['dairy', 'record'])], dtype=object)

Creiamo anche il dizionario di tutte le parole presenti all'interno del nostro corpus di testo, utilizzando la  classe *Dictionary* di gensim. 

In [18]:
from gensim.corpora import Dictionary

dictionary = Dictionary(headlines)

abc.ABCMeta

Filtriamo il dizionario utilizzando il metodo *filter_extremes, per rimuovere le parole che compiano in meno di 10 documenti o in più della metà del corpus di testo, dopodichè teniamo soltanto i 5000 termini più frequenti.

In [0]:
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=5000)

Modello Bag of Words
Codifichiamo le features utilizzando il bag of words, possiamo eseguire il bag of words su di un singolo documento utilizzando il metodo *.doc2bow(doc)* del dizionario, facciamolo per tutti i documenti usando un ciclo.

In [20]:
X = [dictionary.doc2bow(headline) for headline in headlines]
X[0]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]

Creiamo il modello LDA per cercare 10 topic utilizzando la classe *LDAMulticore* di gensim, questa classe da la possibilità di parallelizzare il lavoro su più core della CPU definendo il numero di core da utilizzare all'interno del parametro workers.

In [0]:
from gensim.models import LdaMulticore

lda = LdaMulticore(X, num_topics=10, id2word=dictionary, workers=1)

Utilizzando il parametro *.print_topic()* possiamo stampare le parole più popolari per ogni topic.

In [25]:
for index, topic in lda.print_topics():
    print("\nTOPIC %d - parole più popolari" % (index+1))
    print(topic)


TOPIC 1 - parole più popolari
0.028*"australia" + 0.019*"test" + 0.018*"job" + 0.016*"day" + 0.016*"deal" + 0.013*"ban" + 0.012*"new" + 0.011*"offer" + 0.010*"fight" + 0.009*"action"

TOPIC 2 - parole più popolari
0.032*"win" + 0.016*"force" + 0.015*"lose" + 0.014*"drug" + 0.011*"home" + 0.011*"hop" + 0.010*"appeal" + 0.010*"launch" + 0.009*"campaign" + 0.009*"new"

TOPIC 3 - parole più popolari
0.016*"school" + 0.015*"market" + 0.014*"set" + 0.013*"push" + 0.012*"time" + 0.012*"open" + 0.012*"lead" + 0.012*"year" + 0.011*"record" + 0.010*"share"

TOPIC 4 - parole più popolari
0.043*"man" + 0.038*"charge" + 0.034*"court" + 0.025*"face" + 0.021*"murder" + 0.021*"accuse" + 0.014*"hit" + 0.014*"coast" + 0.014*"shoot" + 0.014*"close"

TOPIC 5 - parole più popolari
0.018*"warn" + 0.017*"rural" + 0.015*"pm" + 0.014*"south" + 0.014*"flood" + 0.013*"north" + 0.012*"residents" + 0.012*"national" + 0.011*"storm" + 0.011*"west"

TOPIC 6 - parole più popolari
0.022*"world" + 0.021*"hospital" + 0.

## Modello TF-IDF
Adesso proviamo con una codifica TF-IDF, possiamo eseguirla utilizzando la classe *TfidfModel* di gensim alla quale dobbiamo passare le features già codificate usando il bag of words.

In [0]:
from gensim.models import TfidfModel

tfidf = TfidfModel(X) # fit
X = tfidf[X] # tranform

Riaddestriamo il modello

In [0]:
lda = LdaMulticore(X, num_topics=10, id2word=dictionary, workers=4)

e stampiamo le parole più popolari per i 10 topic.

In [31]:
for index, topic in lda.print_topics():
    print("\nTOPIC %d - parole più popolari" % (index+1))
    print(topic)


TOPIC 1 - parole più popolari
0.010*"police" + 0.009*"miss" + 0.008*"man" + 0.007*"search" + 0.006*"asylum" + 0.005*"death" + 0.005*"driver" + 0.005*"guilty" + 0.005*"attack" + 0.004*"drink"

TOPIC 2 - parole più popolari
0.007*"plan" + 0.005*"water" + 0.005*"govt" + 0.005*"murray" + 0.004*"hill" + 0.004*"new" + 0.004*"council" + 0.003*"urge" + 0.003*"light" + 0.003*"development"

TOPIC 3 - parole più popolari
0.012*"charge" + 0.012*"court" + 0.010*"man" + 0.007*"assault" + 0.007*"murder" + 0.007*"face" + 0.007*"sex" + 0.006*"sentence" + 0.006*"police" + 0.005*"appeal"

TOPIC 4 - parole più popolari
0.008*"price" + 0.005*"sport" + 0.005*"say" + 0.005*"flood" + 0.005*"abc" + 0.004*"govt" + 0.004*"plan" + 0.004*"telstra" + 0.004*"fund" + 0.004*"new"

TOPIC 5 - parole più popolari
0.016*"interview" + 0.012*"rural" + 0.010*"kill" + 0.007*"national" + 0.006*"news" + 0.006*"nrn" + 0.006*"bomb" + 0.006*"blast" + 0.005*"injure" + 0.005*"iraq"

TOPIC 6 - parole più popolari
0.006*"protest" + 0

## Visualizzare il modello
Possiamo usare pyLDAvis per visualizzare anche un modello LDA creato con gensim, in questo caso la creazione della visualizzazione richiederà più tempo. Installiamo pyLDAvis.

In [32]:
!pip install pyldavis

    100% |████████████████████████████████| 1.6MB 7.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


A differenza di quanto fatto per sklearn, il terzo parametro del modello dovrà essere il dizionaro.

In [33]:
import pyLDAvis.gensim

lda_viz = pyLDAvis.gensim.prepare(lda, X, dictionary, mds='tsne')
pyLDAvis.display(lda_viz)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
